# Clustering examples

This notebook is for developing the fuzzy clustering package and demonstrating how to use it with scikit-learn.

The basic idea is that we create some scikit-learn compatible clustering estimators and a group of scoring functions. They can then be thrown about using scikit-learn for the following important purposes.

* [Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) : Chain together pre-processing, clustering and scoring steps into one object
* [Model evaluation](https://scikit-learn.org/stable/model_selection.html#model-selection) : Cross validation of models to find optimum fitting parameters as scored against various scoring functions

We aim to build clustering estimators and store them in ./Models and maybe add:

* Ensemble methods (run many times to assess stability)
* I/O helper functions (read csv, netcdf, whatever)
* Suite of scoring metrics (fuzzy silouette score, partition coefficient, etc)
* Default, pretrained models (one for each common dataset, like OLCI, OC-CCI etc)

in a way that is generalised to work on all models. Must be a scikit-learn compatible object and must not (!) reinvent wheels here

In [1]:
# link to our custom estimator objects
import sys
sys.path.append('../Estimators/')
sys.path.append('../utils/')
from cmeans_python import CmeansModel
import cluster_scoring

In [19]:
# for serialization of arbitrary Python objects
import pickle
import joblib

# scikit-learn objects for many things
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances, make_scorer, silhouette_score
from sklearn.metrics.cluster import calinski_harabasz_score, davies_bouldin_score, silhouette_samples, silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV


# for data handling and visualization
import pandas as pd
import hvplot.pandas
import xarray as xr
import hvplot.xarray
import holoviews as hv
import numpy as np
from functools import reduce

Create a dummy data set

In [20]:
blobs, labels = make_blobs(n_samples=2000, n_features=2)
dfb = pd.DataFrame(blobs, columns=['x','y'])

In [21]:
dfb.hvplot(x='x',y='y', c=labels, cmap='rainbow', kind="scatter", alpha=0.5)

:Scatter   [x]   (y,_color)

# 1 using cluster estimator

In [22]:
# create a instance of cmeans model, with number of clusters c
cmeans = CmeansModel(c=3)

In [23]:
# fit against the dataset, this generates a cluster set
cmeans.fit(dfb)

CmeansModel(c=3)

In [24]:
# plot the classified data
dfb.hvplot(kind='scatter',x='x',y='y',c=cmeans.labels_, cmap='rainbow') * hv.Points(cmeans.cntr_)

:Overlay
   .Scatter.I :Scatter   [x]   (y,_color)
   .Points.I  :Points   [x,y]

# 2 Performance metrics

if an algorithm comes with its own scoring metric that will be class specific. However, we can get some metrics that are applicable across all methods. 

Scikit-learn has a [suite of metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)

Probably the best way is to make a scorer object from a scoring function with `sklearn.metrics.make_scorer` which can be placed at after clustering in a pipeline which is then fed to a GridSearch like object.

In [25]:
ss = silhouette_samples(dfb, cmeans.labels_)
hv.Bars(
    np.hstack([sorted(ss[cmeans.labels_==i]) for i in range(3)]),
).opts(title=f"silhouette_score = {np.round(silhouette_score(dfb, cmeans.labels_), 10)}")

:Bars   [x]   (y)

# 3 Pipelines: chaining transforms and estimators together

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

https://scikit-learn.org/stable/modules/compose.html

In [26]:
# easy to make a pipeline with whatever pieces you want
pl = make_pipeline(
    StandardScaler(),
    PCA(),
    CmeansModel()     
)

In [27]:
pl.fit(dfb)

Pipeline(steps=[('standardscaler', StandardScaler()), ('pca', PCA()),
                ('cmeansmodel', CmeansModel())])

### 3.x inverse_transforms to get final results in original space

Each transform step in a pipeline has an inverse_transform method. Enabling us to reverse the trasform so that final results can be projected into the original space.

In this example, scaling of the data and principal component analysis are applied before clustering. Therefore the clusters are defined in a transformed space. Using inverse_transforms we regain the orginal x,y space the data came in.

In [28]:
df_cntr = pd.DataFrame(
    pl['standardscaler'].inverse_transform(
        pl['pca'].inverse_transform(
            pl['cmeansmodel'].cntr_
        )
    ),
    columns=['x','y']
)

In [29]:
dfb.hvplot(kind='scatter',x='x',y='y', alpha=0.5, s=1) * \
df_cntr.reset_index().hvplot(kind='scatter',x='x',y='y',c='index', cmap='rainbow', s=30)

:Overlay
   .Scatter.I  :Scatter   [x]   (y)
   .Scatter.II :Scatter   [x]   (y,index)

# 4 GridSearchCV: Exhaustive search to find best fitting parameters

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

One problem we have is that the clustering algorithms performance depends on the fitting parameters (hyper paramters). These are not learnt, rather they determine the behaviour of the algorithm when finding the learnt parameters. GridSearchCV helps by exhaustively fitting the model to the data for every combination of fitting paramters supplied to it. Furthermore, it fits and scores the model to 5 subsets of the data (by default) so that the variance of the score can be measured.

In [33]:
# define a dict of parameters to go through all combinations of values.
# double underscore joins the step name to its parameter
param_grid = {
    'cmeansmodel__c': [2,4,6,8,10],
    'cmeansmodel__m':[1.5,2.0,2.5],
}

In [34]:
# define the scoring metrics you want to evaluate
scoring = {
    'XB': cluster_scoring.xie_beni,
    'SIL': cluster_scoring.hard_silouette,
    'FPC': cluster_scoring.fuzzy_partition_coef,
#     'CH': cluster_scoring.calinski_harabasz,
    'DB': cluster_scoring.davies_bouldin,
}

In [35]:
# create a grid search cross validation object
gs = GridSearchCV(
    pl,
    param_grid=param_grid,
    scoring=scoring,
    refit='XB',
    n_jobs=4
)

In [36]:
# fitting syntax is just like before. Except now it tries all combinations of parameters,
# scores and refits witht he best according to the chosen refit metric.
gs.fit(dfb)

GridSearchCV(estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('cmeansmodel', CmeansModel())]),
             n_jobs=4,
             param_grid={'cmeansmodel__c': [2, 4, 6, 8, 10],
                         'cmeansmodel__m': [1.5, 2.0, 2.5]},
             refit='XB',
             scoring={'DB': <function davies_bouldin at 0x7fa4c9885dd0>,
                      'FPC': <function fuzzy_partition_coef at 0x7fa4c9885cb0>,
                      'SIL': <function hard_silouette at 0x7fa4c9885c20>,
                      'XB': <function xie_beni at 0x7fa4c7e5bc20>})

In [37]:
# create a pandas.DataFrame that contains the results of the fitting
dfgs = pd.DataFrame(gs.cv_results_)

## 5.x plotting 

In [38]:
reduce(lambda a,b:a*b,[dfgs.hvplot.errorbars(
        col='param_cmeansmodel__m',
        row=None,
        x='param_cmeansmodel__c',
        y=f'mean_test_{x}',
        yerr1=f'std_test_{x}',
    )*dfgs.hvplot(
        col='param_cmeansmodel__m',
        row=None,
        kind='scatter',
        x='param_cmeansmodel__c',
        y=f'mean_test_{x}',
        label=f'{x}',
        legend=True,
        ylim=[-1.5,1.5],
        ylabel="score",
        xlabel=r"# clusters c",
) for x in scoring.keys()]).opts(title="3 blobs, 2000 points", show_legend=True) \
+ \
dfb.hvplot(
    kind='scatter',
    x='x',
    y='y',
    c=gs.best_estimator_['cmeansmodel'].labels_,
    cmap='rainbow',
    width=400,
    s=0.1
)

:Layout
   .GridSpace.I :GridSpace   [param_cmeansmodel__m]
      :Overlay
         .ErrorBars.I   :ErrorBars   [param_cmeansmodel__c]   (mean_test_XB,std_test_XB)
         .Scatter.XB    :Scatter   [param_cmeansmodel__c]   (mean_test_XB)
         .ErrorBars.II  :ErrorBars   [param_cmeansmodel__c]   (mean_test_SIL,std_test_SIL)
         .Scatter.SIL   :Scatter   [param_cmeansmodel__c]   (mean_test_SIL)
         .ErrorBars.III :ErrorBars   [param_cmeansmodel__c]   (mean_test_FPC,std_test_FPC)
         .Scatter.FPC   :Scatter   [param_cmeansmodel__c]   (mean_test_FPC)
         .ErrorBars.IV  :ErrorBars   [param_cmeansmodel__c]   (mean_test_DB,std_test_DB)
         .Scatter.DB    :Scatter   [param_cmeansmodel__c]   (mean_test_DB)
   .Scatter.I   :Scatter   [x]   (y,_color)

# 5 serializing models

If we like our results and we want to save them. There exist a few choices;

* pickle / joblib : stores arbitrary python objects in instruction orientated file. Insecure on loading, not cross platform nor cross version supported. Only open trusted files and best used short term.
* custom serialization : pipeline parameters can be stored to netcdf using utils/serialize_models. But it also isn't cross platform nor cross version supported at present.

## 5.1 pickle/joblib

In [39]:
# store a model as a pickle file
pickle.dump(
    pl,
    open(
        "practice_fitted_pipeline.p",
        "wb"
    )
)

# 6 Application to OLCI data

In [ ]:
df = xr.open_dataset("../data/OLCI_RRS_spectral_library_968349_pixels_from_6000_random_G25_300m_files.nc").to_dataframe()

In [ ]:
df = df.reset_index().pivot_table(index='pixel',columns='wavelength',values='Rrs')

In [ ]:
pl.fit(df)

In [ ]:
df_cntr = pd.DataFrame(
    pl['standardscaler'].inverse_transform(
        pl['pca'].inverse_transform(
            pl['cmeansmodel'].cntr_
        )
    ),
)

In [ ]:

dfb.hvplot(kind='scatter',x='x',y='y', alpha=0.5, s=1) * \
df_cntr.reset_index().hvplot(kind='scatter',x='x',y='y',c='index', cmap='rainbow', s=30)